# Weaviate 快速上手示例

本 Notebook 演示如何 **连接本地 Docker 中运行的 Weaviate（REST 端口 8080）**，  
并完成以下操作：

1. 连接测试（含两种连接方式，解决 gRPC 端口问题）  
2. 创建最简单的 `Document` Collection（向量器设为 `none`，手动写入向量）  
3. 使用 Sentence Transformers 生成向量并批量插入样例数据  
4. 进行向量检索 (`near_vector`) 与属性过滤检索 (`where`)  
5. 给出常见问题与清理资源的示例  

> **前置条件**  
> - Docker 中的 Weaviate 已映射端口：  
>   - REST: `8080:8080`  
>   - （可选）gRPC: `50051:50051` —— 如果未映射，请使用 *方式 B* 连接；  
> - Python ≥3.9 环境，并可 `pip install`；  
> - 如需使用内置 `text2vec-openai` 向量器，请先在环境变量里设置 `OPENAI_API_KEY`，然后可去掉手动向量代码。  


In [17]:
import weaviate, json

client = weaviate.connect_to_local(host="localhost", port=8080, grpc_port=50051)

print("\n=== 当前 Collections ===")
for col_meta in client.collections.list_all():   # 返回 List[CollectionMeta]
    name   = col_meta.name
    dim    = col_meta.vector_index_config.dimensions
    vecmod = col_meta.vectorizer_config.vectorizer
    collection = client.collections.get(name)

    total = collection.data.count()
    print(f"- {name}  | 对象 {total} 条 | 维度 {dim} | vectorizer={vecmod}")

    # 抽样 2 条看看字段
    sample = collection.query.fetch_objects(limit=2)
    for obj in sample.objects:
        print("  •", json.dumps(obj.properties, ensure_ascii=False))



=== 当前 Collections ===


In [ ]:
import weaviate

# 方式 A：已映射 gRPC 端口 50051（推荐）
client = weaviate.connect_to_local(
    host="localhost",
    port=8080,

)

# 如果 **没有** 映射 50051，可使用跳过启动检查的方式 B：
# client = weaviate.connect_to_local(
#     host="localhost",
#     port=8080,
#     skip_init_checks=True
# )

print("Weaviate 是否就绪:", client.is_ready())

Weaviate 是否就绪: True


/home/kapibala/miniconda3/envs/evennia/lib/python3.12/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/tmp/ipykernel_1519208/4196431880.py:4: ResourceWarning: unclosed <socket.socket fd=77, family=2, type=1, proto=6, laddr=('127.0.0.1', 33948), raddr=('127.0.0.1', 8080)>
  client = weaviate.connect_to_local(


In [19]:
import weaviate.classes.config as wvc

COLLECTION = "Document"

if not client.collections.exists(COLLECTION):
    client.collections.create(
        name=COLLECTION,
        properties=[
            wvc.Property(name="title", data_type=wvc.DataType.TEXT),
            wvc.Property(name="text", data_type=wvc.DataType.TEXT),
        ],
        vectorizer_config=wvc.Configure.Vectorizer.none()  # 手动向量
    )
    print(f"已创建 Collection: {COLLECTION}")
else:
    print(f"Collection '{COLLECTION}' 已存在")

已创建 Collection: Document


/home/kapibala/miniconda3/envs/evennia/lib/python3.12/site-packages/weaviate/collections/classes/config.py:1975: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


In [20]:
import os
from langchain_community.embeddings import ZhipuAIEmbeddings

# ① 初始化智谱 Embedding 对象
embedder = ZhipuAIEmbeddings(
    model="embedding-3",     # 智谱最新 text2vec 模型
    dimensions=1024,          # 可选：1024/768/512，默认 1024
    api_key="8725cd11e5cc4c56aabc5142442e59f6.IceSidZVpsp39Mj3"  # 如未设置 env，可主动传
)

# ② 准备待写入数据
docs = [
    {"title": "Weaviate Quickstart",
     "text": "Weaviate is a vector database built to scale your ML applications."},
    {"title": "Evennia MUD Engine",
     "text": "Evennia is an open-source Python library for creating online text games."},
    {"title": "Python Dataclasses",
     "text": "Dataclasses make it easier to create classes for storing data in Python."}
]

collection = client.collections.get(COLLECTION)

# ③ 生成向量并写入 Weaviate
for d in docs:
    vec = embedder.embed_query(d["text"])   # 单条文本 → 1024 维向量
    collection.data.insert(properties=d, vector=vec)

print("样例数据已插入，共:",
      collection.aggregate.over_all().total_count)


/tmp/ipykernel_1519208/26139997.py:21: ResourceWarning: unclosed <socket.socket fd=65, family=2, type=1, proto=6, laddr=('127.0.0.1', 49394), raddr=('127.0.0.1', 8080)>
  collection = client.collections.get(COLLECTION)


样例数据已插入，共: 3


In [6]:
from weaviate.classes.query import Filter

# ① 构造过滤器：title 字段包含 “Python”
title_has_python = Filter.by_property("title").contains_any(["Python"])

# ② 取前 5 条匹配对象
resp = collection.query.fetch_objects(
    filters=title_has_python,
    limit=5
)

print("含有 'Python' 的条目：")
for obj in resp.objects:
    print("•", obj.properties["title"])


NameError: name 'collection' is not defined

In [12]:
import weaviate, json, textwrap

client = weaviate.connect_to_local(host="localhost",
                                   port=8080,
                                   grpc_port=50051)

print("=== 当前 Collections ===")
for name in client.collections.list_all():           # 4.x: list[str]
    col = client.collections.get(name)

    # 1) 对象总数
    total = col.aggregate.over_all().total_count

    # 2) 取第一条对象看向量长度 → 维度
    dim = "?"
    try:
        obj = col.query.fetch_objects(limit=1, include_vector=True).objects[0]
        if obj.vector is not None:
            dim = len(obj.vector)
    except IndexError:
        dim = "0"

    print(f"- {name:<20}  对象: {total:<5}  维度: {dim}")

    # 3) 再抽 2 条示例（不带向量，防止输出过大）
    sample = col.query.fetch_objects(limit=20, include_vector=False)
    for o in sample.objects:
        preview = textwrap.shorten(o.properties.get("content", ""), width=60, placeholder="…")
        print(f"  • {preview}")
    print()


=== 当前 Collections ===
- BattleKnowledge       对象: 20     维度: 1
  • 【剑法招式】 - 一剑封喉：迅疾如电的致命一击，专攻要害 - 剑气纵横：以内力催动剑气，远程攻击 -…
  • ### 费德勒简介 罗杰·费德勒（Roger Federer，1981 年 8 月 8…
  • > **武器推荐**： > *传奇拍* **“Pro Staff RF97”** ——…
  • 【攻击效果描述】 - 命中要害：招式精准击中敌人弱点 - 力量爆发：全力一击展现强大威力 -…
  • 【剑法招式】 - 一剑封喉：迅疾如电的致命一击，专攻要害 - 剑气纵横：以内力催动剑气，远程攻击 -…
  • 【剑客特征】 - 身法飘逸，如风中柳絮 - 剑光闪烁，寒光逼人 - 气质超然，有仙风道骨 - 出手果断，招招致命…
  • 【攻击效果描述】 - 命中要害：招式精准击中敌人弱点 - 力量爆发：全力一击展现强大威力 -…
  • | 招式名称 | 类型 | 灵感来源 | 效果示例（数值可按你游戏平衡调整） | | ------------ |…
  • 【攻击效果描述】 - 命中要害：招式精准击中敌人弱点 - 力量爆发：全力一击展现强大威力 -…
  • | 招式名称 | 类型 | 灵感来源 | 效果示例（数值可按你游戏平衡调整） | | ------------ |…
  • 【剑法招式】 - 一剑封喉：迅疾如电的致命一击，专攻要害 - 剑气纵横：以内力催动剑气，远程攻击 -…
  • * 投资并深度参与瑞士运动品牌 **On Running**，协助设计 “The ROGER Pro” 鞋款；公司…
  • 【剑客特征】 - 身法飘逸，如风中柳絮 - 剑光闪烁，寒光逼人 - 气质超然，有仙风道骨 - 出手果断，招招致命…
  • 【攻击效果描述】 - 命中要害：招式精准击中敌人弱点 - 力量爆发：全力一击展现强大威力 -…
  • 【剑客特征】 - 身法飘逸，如风中柳絮 - 剑光闪烁，寒光逼人 - 气质超然，有仙风道骨 - 出手果断，招招致命…
  • > **武器推荐**： > *传奇拍* **“Pro Staff RF97”** ——…
  • ### 费德勒简介 罗杰·费德勒（Roger Federer，198

/tmp/ipykernel_1560674/2024230957.py:9: ResourceWarning: unclosed <socket.socket fd=75, family=2, type=1, proto=6, laddr=('127.0.0.1', 60870), raddr=('127.0.0.1', 8080)>
  col = client.collections.get(name)


In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import weaviate
import textwrap
from langchain_community.embeddings import ZhipuAIEmbeddings

def search_federer_context():
    """使用费德勒描述在BattleKnowledge collection中查找相关上下文"""
    
    # 连接到本地Weaviate实例
    client = weaviate.connect_to_local(
        host="localhost", 
        port=8080, 
        grpc_port=50051
    )
    
    try:
        # 初始化智谱AI embedding模型（与notebook中保持一致）
        embedder = ZhipuAIEmbeddings(
            model="embedding-3",
            dimensions=1024,
            api_key="8725cd11e5cc4c56aabc5142442e59f6.IceSidZVpsp39Mj3"
        )
        
        # 查询文本
        query_text = " 罗杰·费德勒（Roger Federer，1981 年生）被誉为史上最杰出的男子网球选手之一，职业生涯共夺得 20 座大满贯单打冠军、103 个 ATP 单打冠军，并累积 310 周世界第一（其中连续 237 周创纪录）他于 2022 年正式退役后，仍活跃于网球及公益领域：继续担任拉沃尔杯的联合创办人兼全球大使，并通过“罗杰·费德勒基金会”资助南部非洲与瑞士的儿童教育项目。2025 年，他还受邀为传奇赛事——勒芒 24 小时耐力赛挥动起跑旗，延续跨界影响力 This is User #1. 剑 剑 战斗 攻击 招式 武器"
        
        print("=== 搜索查询 ===")
        print(f"查询文本: {query_text}")
        print()
        
        # 生成查询向量
        query_vector = embedder.embed_query(query_text)
        print(f"查询向量维度: {len(query_vector)}")
        print()
        
        # 获取BattleKnowledge collection
        collection = client.collections.get("BattleKnowledge")
        
        # 执行向量相似性搜索
        response = collection.query.near_vector(
            near_vector=query_vector,
            limit=5,  # 返回最相关的5个结果
            return_metadata=["score"]  # 返回相似度分数
        )
        
        print("=== 搜索结果 ===")
        print(f"找到 {len(response.objects)} 个相关结果:\n")
        
        for i, obj in enumerate(response.objects, 1):
            # 获取相似度分数
            print(vars(obj.metadata))
            score = obj.metadata.score if obj.metadata and obj.metadata.score else "N/A"
            
            print(f"--- 结果 {i} (相似度: {score}) ---")
            
            # 获取内容并格式化显示
            content = obj.properties.get("content", "无内容")
            
            # 使用textwrap格式化长文本
            wrapped_content = textwrap.fill(content, width=80, 
                                          initial_indent="  ", 
                                          subsequent_indent="  ")
            print(wrapped_content)
            print()
        
        # 显示collection的总体信息
        total_objects = collection.aggregate.over_all().total_count
        print(f"=== Collection信息 ===")
        print(f"BattleKnowledge collection总对象数: {total_objects}")
        print("搜索完成！")
        
    except Exception as e:
        print(f"搜索过程中出现错误: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # 关闭连接
        client.close()

if __name__ == "__main__":
    search_federer_context()

=== 搜索查询 ===
查询文本:  罗杰·费德勒（Roger Federer，1981 年生）被誉为史上最杰出的男子网球选手之一，职业生涯共夺得 20 座大满贯单打冠军、103 个 ATP 单打冠军，并累积 310 周世界第一（其中连续 237 周创纪录）他于 2022 年正式退役后，仍活跃于网球及公益领域：继续担任拉沃尔杯的联合创办人兼全球大使，并通过“罗杰·费德勒基金会”资助南部非洲与瑞士的儿童教育项目。2025 年，他还受邀为传奇赛事——勒芒 24 小时耐力赛挥动起跑旗，延续跨界影响力 This is User #1. 剑 剑 战斗 攻击 招式 武器

查询向量维度: 1024

=== 搜索结果 ===
找到 5 个相关结果:

{'creation_time': None, 'last_update_time': None, 'distance': None, 'certainty': None, 'score': 0.0, 'explain_score': None, 'is_consistent': None, 'rerank_score': None}
--- 结果 1 (相似度: N/A) ---
  ### 费德勒简介  罗杰·费德勒（Roger Federer，1981 年 8 月 8 日生于瑞士巴塞尔）被誉为网坛的“优雅大师”。自 1998
  年转入职业赛场起，他以流畅的一手单反、精准的发球落点和行云流水的步伐征服观众，累计 20 座大满贯男单冠军、310 周世界第一（其中 237
  周连续），并与纳达尔、德约科维奇共同缔造了“Big 3”时代。  * **创办 Laver Cup**：2017
  年起担任赛事联席创办人兼股东，退役后仍是推广大使，2025 年赛事将在旧金山举行。([lavercup.com][1]) * **传奇谢幕**：2022 年
  9 月 23 日伦敦 O₂ 体育馆，费德勒在 Laver Cup 与纳达尔搭档完成最后一战，赛后“泪别”场面震撼全球。([lavercup.com][2])
  * **退役后的舞台**

{'creation_time': None, 'last_update_time': None, 'distance': None, 'certainty': None, 'score': 